In [ ]:
import sys as sys 
sys.path.append('..')

import numpy as np 
import pandas as pd
from ModelTools import MeanRegression
import plotnine as gg


### 通过控制得分函数的方法来选择不同的模型

In [ ]:
n = 500
rng = np.random.default_rng(0)
x = rng.normal(size=n,loc=0,scale=0.5)
y = rng.lognormal(mean=5*np.cos(x),sigma=0.5)
df = pd.DataFrame(data={'x':x,'y':y,'ts':pd.date_range(start='2000-01-01',periods=n,freq='min')})

fit_mse = MeanRegression(data=df,col_x=['x'],col_y='y',exp_model=False,cv_score='mse')
fit_mse.fit().fit_final_model()

fit_mdae = MeanRegression(data=df,col_x=['x'],col_y='y',exp_model=False,cv_score='mdae')
fit_mdae.fit().fit_final_model()

fit_mape = MeanRegression(data=df,col_x=['x'],col_y='y',exp_model=False,cv_score='mape')
fit_mape.fit().fit_final_model()
gg.options.figure_size = [15,5]
(
    gg.ggplot(data=df)
    +gg.aes(x='x',y=y)
    +gg.geom_point()
    +gg.geom_line(gg.aes(y=fit_mse.predict(),color="'mse'"),size=1)
    +gg.geom_line(gg.aes(y=fit_mdae.predict(),color="'mdae'"),size=1)
    +gg.geom_line(gg.aes(y=fit_mape.predict(),color="'mape'"),size=1)
)

In [ ]:
n = 500
rng = np.random.default_rng(0)

x1 = rng.normal(size=n,loc=0,scale=1)
x2 = rng.normal(size=n,loc=0,scale=1)
y = rng.normal(loc=x2+5,scale=0.5)
# y = rng.lognormal(mean=x2,sigma=0.5)

df = pd.DataFrame(data={'x1':x1,'x2':x2,'y':y,'ts':pd.date_range(start='2000-01-01',periods=n,freq='min')})

# print(df)

m = MeanRegression(data=df,col_x=['x2'],col_y='y',col_ts='ts',ts_freq='min',exp_model=False)
m.fit(base=['lm'],best_model='OLS',best_model_only=True).fit_final_model()

In [ ]:
m.final_model['OLS'].intercept_

In [ ]:
import plotnine as gg

(
    gg.ggplot(data=df)
    +gg.aes(x='x2',y=y)
    +gg.geom_point()
    +gg.geom_line(gg.aes(y=m.predict()),color='red',size=1)
)

In [ ]:
m.fit_final_model()
x = m.predict_interval(type='confidence',n_bootstrap=1000)
(
    pd.DataFrame(x)
    .assign(x=x2,mean=m.predict(),real=np.cos(x2),y=y)
    .pipe(gg.ggplot)
    +gg.aes(x='x')
    + gg.geom_point(gg.aes(y='y'),alpha=0.1)
    +gg.geom_line(gg.aes(y='real'),color='red')
    +gg.geom_line(gg.aes(y='down'),color='green')
    +gg.geom_line(gg.aes(y='high'),color='green')
)

In [ ]:
n = 500
rng = np.random.default_rng(0)
x1 = rng.uniform(-3,3,size=n)
# y = rng.lognormal(mean=x1,sigma=0.1)
y = rng.normal(loc=np.cos(x1)+1,scale=0.2)
# y = rng.normal(loc=np.cos(x1)+1,scale=(x1-min(x1))/np.ptp(x1)+0.01)
df = pd.DataFrame(data={'x1':x1,'y':y})

# print(df)

m = MeanRegression(data=df,col_x=['x1'],col_y='y',exp_model=False)
m.fit(base=['lm'])
m.fit_final_model()

x = m.predict_interval(type='predict',n_bootstrap=100,alpha=0.01)
(
    pd.DataFrame(x)
    .assign(x=x1,m=m.predict(),real=2*x1,y=y)
    .pipe(gg.ggplot)
    +gg.aes(x='x')
    + gg.geom_point(gg.aes(y='y'),alpha=0.1)
    # +gg.geom_line(gg.aes(y='real'),color='red')
    +gg.geom_line(gg.aes(y='m'),color='yellow')
    +gg.geom_line(gg.aes(y='down'),color='green')
    +gg.geom_line(gg.aes(y='high'),color='blue')
)